In [ ]:
import torch

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!7z x "/content/drive/MyDrive/Chicago_orig.7z" -o/content/Chicago_orig



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/                                 1 file, 1881240717 bytes (1795 MiB)

Extracting archive: /content/drive/MyDrive/Chicago_orig.7z
--
Path = /content/drive/MyDrive/Chicago_orig.7z
Type = 7z
Physical Size = 1881240717
Headers Size = 1107
Method = LZMA2:24
Solid = +
Blocks = 2

  0%      0% 9 - Chicago_orig/train/gt/chicago13.tif                                              0% 15 - Chicago_orig/train/gt/chicago19.tif                                               0% 22 - Chicago_orig/

In [3]:
import os
import torch
import torch.nn as nn

##############################################################

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class DownSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = DoubleConv(in_channels, out_channels)
        self.pool = nn.MaxPool2d(2)

    def forward(self, x):
        skip = self.conv(x)
        down = self.pool(skip)
        return skip, down

class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        # in_channels: сколько пришло снизу (например, 1024)
        # out_channels: сколько каналов хотим после апсэмплинга (512)
        self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        # После concat будет out_channels (от апсэмпла) + out_channels (от скипа) = out_channels * 2
        self.double_conv = DoubleConv(out_channels * 2, out_channels)

    def forward(self, x, skip):
        x = self.up(x)
        x = torch.cat([x, skip], dim=1) # Склеиваем по каналам
        return self.double_conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=1):
        super().__init__()
        # Энкодер
        self.down1 = DownSample(in_channels, 64)
        self.down2 = DownSample(64, 128)
        self.down3 = DownSample(128, 256)
        self.down4 = DownSample(256, 512)

        # Центр (Bottleneck)
        self.bottleneck = DoubleConv(512, 1024)

        # Декодер
        # Теперь передаем: сколько каналов НА ВХОДЕ в апсэмпл и сколько НА ВЫХОДЕ из всего блока
        self.up1 = UpSample(1024, 512)
        self.up2 = UpSample(512, 256)
        self.up3 = UpSample(256, 128)
        self.up4 = UpSample(128, 64)

        self.out = nn.Conv2d(64, num_classes, kernel_size=1)

    def forward(self, x):
        sk1, x = self.down1(x)
        sk2, x = self.down2(x)
        sk3, x = self.down3(x)
        sk4, x = self.down4(x)

        x = self.bottleneck(x)

        x = self.up1(x, sk4)
        x = self.up2(x, sk3)
        x = self.up3(x, sk2)
        x = self.up4(x, sk1)

        return self.out(x)

In [4]:
# 1. Определяем устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [5]:
# 2. Создаем модель и сразу отправляем на устройство
model = UNet().to(device)

In [6]:
#3. Загружаем чекпоинт
# map_location гарантирует, что веса загрузятся правильно на CPU или GPU
checkpoint = torch.load(f"/content/drive/MyDrive/checkpoint_epoch_110.pth", map_location=device)
model.load_state_dict(checkpoint)
print("Модель загружена.")

Модель загружена.


In [7]:
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt

In [8]:
def predict_full_image(model, image_path, mask_path, device, patch_size=256):
    # 1. Загрузка оригинального фото и маски
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Загружаем истинную маску (GT)
    gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if gt_mask is None:
        raise FileNotFoundError(f"Маска не найдена по пути: {mask_path}")

    h, w, _ = image.shape  # 5000, 5000

    # --- РАСЧЕТ ПЛОЩАДИ ПО ИСТИННОЙ МАСКЕ (GT) ---
    # В масках Inria здания — это 255, фон — 0.
    # Делаем бинарную маску (0 и 1)
    gt_binary = (gt_mask > 0).astype(np.uint8)
    gt_pixel_count = np.sum(gt_binary)
    gt_area_m2 = gt_pixel_count * 0.09  # 0.3м * 0.3м = 0.09
    # ----------------------------------------------

    # 2. Padding (Дополнение до 5120 для модели)
    pad_h = (patch_size - h % patch_size) % patch_size
    pad_w = (patch_size - w % patch_size) % patch_size
    padded_img = cv2.copyMakeBorder(image, 0, pad_h, 0, pad_w, cv2.BORDER_CONSTANT, value=0)

    # 3. Подготовка холста для предсказаний
    full_pred = np.zeros((h + pad_h, w + pad_w), dtype=np.float32)

    model.eval()
    # Константы нормализации
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])

    with torch.no_grad():
        for y in range(0, padded_img.shape[0], patch_size):
            for x in range(0, padded_img.shape[1], patch_size):
                patch = padded_img[y:y + patch_size, x:x + patch_size]

                # Подготовка патча
                input_patch = patch / 255.0
                input_patch = (input_patch - mean) / std
                input_patch = torch.from_numpy(input_patch).permute(2, 0, 1).float().unsqueeze(0).to(device)

                # Предсказание
                output = model(input_patch)
                pred = torch.sigmoid(output).squeeze().cpu().numpy()
                full_pred[y:y + patch_size, x:x + patch_size] = pred

    # 4. Обрезка и бинаризация предсказания
    final_pred_mask = full_pred[:h, :w]
    binary_pred = (final_pred_mask > 0.5).astype(np.uint8)

    # --- РАСЧЕТ ПЛОЩАДИ ПО ПРЕДСКАЗАНИЮ МОДЕЛИ ---
    pred_pixel_count = np.sum(binary_pred)
    pred_area_m2 = pred_pixel_count * 0.09
    # ----------------------------------------------

    # Расчет разницы в процентах (ошибка)
    error_percent = abs(pred_area_m2 - gt_area_m2) / gt_area_m2 * 100

    return image, gt_mask, binary_pred, pred_area_m2, gt_area_m2, error_percent

In [9]:
def plot_final_results(image, gt, pred, area,area_gt,filename):
    fig, ax = plt.subplots(1, 3, figsize=(20, 7))

    # Уменьшаем для отрисовки (например, до 1000x1000)
    preview_img = cv2.resize(image, (1000, 1000))
    preview_gt = cv2.resize(gt, (1000, 1000))
    preview_pred = cv2.resize(pred, (1000, 1000))

    ax[0].imshow(preview_img)
    ax[0].set_title("Оригинальный снимок")
    ax[0].axis('off')

    ax[1].imshow(preview_gt, cmap='gray')
    ax[1].set_title(f"Истинная маска (GT)-Площадь: {area_gt:,.0f} м²")
    ax[1].axis('off')

    ax[2].imshow(preview_pred, cmap='gray')
    ax[2].set_title(f"Предсказание модели-Площадь: {area:,.0f} м²")
    ax[2].axis('off')

    plt.tight_layout()
    plt.show()
    #plt.savefig(f'Ploshad-{filename}.png')

In [10]:
# 1. Задаем пути к папкам '/content/Chicago_orig/train/images'
images_dir = '/content/Chicago_orig/Chicago_orig/train/images'
masks_dir = '/content/Chicago_orig/Chicago_orig/train/gt'

# 2. Выбираем файлы (например, снимки Чикаго с 30 по 36, которые не участвовали в обучении)
#test_files = [f for f in os.listdir(images_dir) if f.startswith('chicago') and f.endswith('.tif')]
#test_files = test_files[30:]  # Берем последние 6 файлов для теста
test_files = ['chicago22.tif', 'chicago23.tif', 'chicago25.tif', 'chicago10.tif', 'chicago16.tif']

In [ ]:
all_areas = {}  # Словарь для хранения площадей

for filename in test_files:
    print(f"Обработка {filename}...")

    img_path = os.path.join(images_dir, filename)
    mask_path = os.path.join(masks_dir, filename)

    # Вызываем функцию (которую мы написали раньше)
    #image, gt, pred, area = predict_full_image(model, img_path, mask_path, device)
    image, gt, pred, area_model, area_gt, error = predict_full_image(model, img_path, mask_path, device)

    # Сохраняем результат в словарь
    all_areas[filename] = [area_model,area_gt,error]

    # Визуализируем (опционально)
    plot_final_results(image, gt, pred, area_model,area_gt,filename)

Обработка chicago22.tif...


In [ ]:
# Вывод итогов
print("\nИтоговые площади застройки:")
print(all_areas)
#for name, val in all_areas.items():
#    print(f"{name}: {val:,.2f} м²")
for name, val in all_areas.items():
    print(f"{name}:")
    print(f"  Модель: {val [0] :,.2f} м²")
    print(f"  Факт:   {val [1] :,.2f} м²")
    print(f"  Ошибка: {val [2] :.2f}%")
    print("-" * 30) # Разделительная линия

# Вывод итогов
print("\nИтоговые площади застройки:")
print(all_areas)
#for name, val in all_areas.items():
#    print(f"{name}: {val:,.2f} м²")
for name, val in all_areas.items():
    print(f"{name}:")
    print(f"  Модель: {val [0] :,.2f} м²")
    print(f"  Факт:   {val [1] :,.2f} м²")
    print(f"  Ошибка: {val [2] :.2f}%")
    print("-" * 30) # Разделительная линия